In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN_RETINA,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
listings = pd.read_csv('/home/novais/Downloads/listings.csv')

In [5]:
listings.head()

,Unnamed: 0,listing_id,host_id,neighbourhood,latitude,longitude,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,lr_m,lr_d,lr_y,lat,lon,location,loc,loc_id
0,0,2384,2613,Hyde Park,41.78886,-87.58671,2,159,2.89,1,306,7,11,2019,41.79,-87.59,"(41.78886, -87.58671)","(41.79, -87.59)",124
1,1,4505,5775,South Lawndale,41.85495,-87.69696,1,373,2.96,1,326,6,25,2019,41.85,-87.70,"(41.85495, -87.69696)","(41.85, -87.7)",190
2,2,7126,17928,West Town,41.90289,-87.68182,1,349,2.86,1,323,6,30,2019,41.90,-87.68,"(41.90289, -87.68182)","(41.9, -87.68)",269
3,3,9811,33004,Lincoln Park,41.91769,-87.63788,1,38,0.59,8,327,6,30,2019,41.92,-87.64,"(41.91769, -87.63788000000001)","(41.92, -87.64)",310
4,4,10610,2140,Hyde Park,41.79708,-87.59195,2,39,0.68,4,247,6,6,2019,41.80,-87.59,"(41.79708, -87.59195)","(41.8, -87.59)",140


In [6]:
listings_locations = listings[['loc','loc_id']]
listings_locations = listings_locations.drop_duplicates('loc_id')
listings_locations = listings_locations.set_index('loc_id')
listings_locations = listings_locations.sort_index()
listings_locations = listings_locations.reset_index()
listings_locations['loc_id2']=listings_locations.index
list_locs=list(listings_locations['loc'])
len(list_locs)

370

In [7]:
listings=listings.merge(listings_locations,on=['loc_id','loc'])
listings.head()

,Unnamed: 0,listing_id,host_id,neighbourhood,latitude,longitude,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,lr_m,lr_d,lr_y,lat,lon,location,loc,loc_id,loc_id2
0,0,2384,2613,Hyde Park,41.78886,-87.58671,2,159,2.89,1,306,7,11,2019,41.79,-87.59,"(41.78886, -87.58671)","(41.79, -87.59)",124,71
1,292,2604454,13339125,Hyde Park,41.78977,-87.58916,1,85,1.42,3,41,7,11,2019,41.79,-87.59,"(41.789770000000004, -87.58915999999999)","(41.79, -87.59)",124,71
2,684,6524346,34121377,Hyde Park,41.79119,-87.59099,2,26,0.52,1,34,7,7,2019,41.79,-87.59,"(41.79119, -87.59099)","(41.79, -87.59)",124,71
3,2618,18549719,47172572,Hyde Park,41.79296,-87.59275,1,127,4.77,60,96,7,2,2019,41.79,-87.59,"(41.79296, -87.59275)","(41.79, -87.59)",124,71
4,3756,22320506,47172572,Hyde Park,41.79386,-87.59469,1,99,5.32,60,93,7,8,2019,41.79,-87.59,"(41.793859999999995, -87.59469)","(41.79, -87.59)",124,71


In [8]:
ids=list(airbnb_cr['listing_id'].unique())

In [9]:
listings=listings[listings.listing_id.isin(ids)]

In [10]:
listings.head()

,Unnamed: 0,listing_id,host_id,neighbourhood,latitude,longitude,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,lr_m,lr_d,lr_y,lat,lon,location,loc,loc_id,loc_id2
0,0,2384,2613,Hyde Park,41.78886,-87.58671,2,159,2.89,1,306,7,11,2019,41.79,-87.59,"(41.78886, -87.58671)","(41.79, -87.59)",124,71
1,292,2604454,13339125,Hyde Park,41.78977,-87.58916,1,85,1.42,3,41,7,11,2019,41.79,-87.59,"(41.789770000000004, -87.58915999999999)","(41.79, -87.59)",124,71
2,684,6524346,34121377,Hyde Park,41.79119,-87.59099,2,26,0.52,1,34,7,7,2019,41.79,-87.59,"(41.79119, -87.59099)","(41.79, -87.59)",124,71
3,2618,18549719,47172572,Hyde Park,41.79296,-87.59275,1,127,4.77,60,96,7,2,2019,41.79,-87.59,"(41.79296, -87.59275)","(41.79, -87.59)",124,71
4,3756,22320506,47172572,Hyde Park,41.79386,-87.59469,1,99,5.32,60,93,7,8,2019,41.79,-87.59,"(41.793859999999995, -87.59469)","(41.79, -87.59)",124,71


In [11]:
listings2=listings.drop_duplicates(subset='location')

In [12]:
listings2['room_type']=listings2['room_type'].astype(str)

In [13]:
listings2['aval'] = listings2['availability_365'].apply([lambda x: "{0:.2%}".format(x/365)])
listings2.head()

,Unnamed: 0,listing_id,host_id,neighbourhood,latitude,longitude,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count,...,lr_m,lr_d,lr_y,lat,lon,location,loc,loc_id,loc_id2,aval
0,0,2384,2613,Hyde Park,41.78886,-87.58671,2,159,2.89,1,...,7,11,2019,41.79,-87.59,"(41.78886, -87.58671)","(41.79, -87.59)",124,71,83.84%
1,292,2604454,13339125,Hyde Park,41.78977,-87.58916,1,85,1.42,3,...,7,11,2019,41.79,-87.59,"(41.789770000000004, -87.58915999999999)","(41.79, -87.59)",124,71,11.23%
2,684,6524346,34121377,Hyde Park,41.79119,-87.59099,2,26,0.52,1,...,7,7,2019,41.79,-87.59,"(41.79119, -87.59099)","(41.79, -87.59)",124,71,9.32%
3,2618,18549719,47172572,Hyde Park,41.79296,-87.59275,1,127,4.77,60,...,7,2,2019,41.79,-87.59,"(41.79296, -87.59275)","(41.79, -87.59)",124,71,26.30%
4,3756,22320506,47172572,Hyde Park,41.79386,-87.59469,1,99,5.32,60,...,7,8,2019,41.79,-87.59,"(41.793859999999995, -87.59469)","(41.79, -87.59)",124,71,25.48%


In [14]:
# colormap = {'1': 'red', '2': 'green', '3': 'blue'}
# colors = [colormap[x] for x in listings2['room_type']]
# listings2['location'] = listings2['location'].apply(lambda x: literal_eval(x))
# listings2['coords_x'] = listings2['location'].apply(lambda x: merc(x)[0])
# listings2['coords_y'] = listings2['location'].apply(lambda x: merc(x)[1])
# p1 = figure(title = "Listings for room type")
# p1.xaxis.axis_label = 'Latitude'
# p1.yaxis.axis_label = 'Longitude'
# source = ColumnDataSource(data={
#     'nr'      : listings2['number_of_reviews'],
#     'cx'      : listings2['coords_x']
#     'cy'      : listings2['coords_y']
# })
# p1.add_tools(HoverTool(
#     tooltips=[
#         ( 'nr', '@volume{0.00 a}'      )
#     ]

# ))
# p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
# p1.add_tile(CARTODBPOSITRON)
# p1.circle(x = listings2['coords_x'],y = listings2['coords_y'],color=colors,source=source)
# p1_script, p1_div = components(p1)

In [15]:
# colormap = {'1': 'red', '2': 'green', '3': 'blue'}
# namemap = {'1':'Quarto 1','2':'Quarto 2','3':'Quarto 3'}

# listings2['location'] = listings2['location'].apply(lambda x: literal_eval(x))
# listings2['coords_x'] = listings2['location'].apply(lambda x: merc(x)[0])
# listings2['coords_y'] = listings2['location'].apply(lambda x: merc(x)[1])

# p1 = figure(title = "Listings for room type")
# p1.xaxis.axis_label = 'Latitude'
# p1.yaxis.axis_label = 'Longitude'

# source = ColumnDataSource(data=dict(
#     nr=listings2['number_of_reviews'],
#     cx=listings2['coords_x'],
#     cy=listings2['coords_y'],
#     colors = [colormap[x] for x in listings2['room_type']],
#     name = [namemap[x] for x in listings2['room_type']] ))





# p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
# p1.add_tile(CARTODBPOSITRON)
# p1.circle(x = 'cx',y = 'cy',color='colors',source=source,legend='name')

# p1.add_tools(HoverTool(
#     tooltips=[
#         ( 'nr', '@nr')
#     ],
    
#     formatters={
#         'nr' : 'numeral',
#     },

# ))

# p1.legend.location = "top_right"
# p1.legend.click_policy="hide"

# p1_script, p1_div = components(p1)

In [16]:
colormap = {'1': '#29788E', '2': '#22A784', '3': '#79D151'}
namemap = {'1':'Entire Home/Apt','2':'Private Room','3':'Shared Room'}

listings2['location'] = listings2['location'].apply(lambda x: literal_eval(x))
listings2['coords_x'] = listings2['location'].apply(lambda x: merc(x)[0])
listings2['coords_y'] = listings2['location'].apply(lambda x: merc(x)[1])

l1 = listings2[listings2['room_type']=='1']
l2 = listings2[listings2['room_type']=='2']
l3 = listings2[listings2['room_type']=='3']

p1 = figure(title = "Listings for room type")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

source1 = ColumnDataSource(data=dict(
    nr=l1['aval'],
    cx=l1['coords_x'],
    cy=l1['coords_y'],
    colors = [colormap[x] for x in l1['room_type']],
    name = [namemap[x] for x in l1['room_type']] ))
source2 = ColumnDataSource(data=dict(
    nr=l2['aval'],
    cx=l2['coords_x'],
    cy=l2['coords_y'],
    colors = [colormap[x] for x in l2['room_type']],
    name = [namemap[x] for x in l2['room_type']] ))
source3 = ColumnDataSource(data=dict(
    nr=l3['aval'],
    cx=l3['coords_x'],
    cy=l3['coords_y'],
    colors = [colormap[x] for x in l3['room_type']],
    name = [namemap[x] for x in l3['room_type']] ))


p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',color='colors',source=source1,size=8,legend='name',line_color="#000000",line_width=0.25)
p1.circle(x = 'cx',y = 'cy',color='colors',source=source2,size=8,legend='name',line_color="#000000",line_width=0.25)
p1.circle(x = 'cx',y = 'cy',color='colors',source=source3,size=8,legend='name',line_color="#000000",line_width=0.25)

p1.add_tools(HoverTool(
    tooltips=[
        ( 'Est. Availability', '@nr')
    ],
    
    formatters={
        'Est. Availability' : 'numeral',
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [17]:
output_notebook()


Loading BokehJS ...

In [19]:
show(p1)

In [ ]:
print(p1_script)

In [ ]:
print(p1_div)